In [1]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/캡스톤/data/beauty_3.zip"

In [5]:
with open("./beauty_3/train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open("./beauty_3/valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open("./beauty_3/test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [6]:
with open("./beauty_3/train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open("./beauty_3/valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open("./beauty_3/test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [7]:
# ID
train_data[0] = tf.convert_to_tensor(train_data[0], dtype=tf.float32)
train_data[1] = tf.convert_to_tensor(train_data[1], dtype=tf.float32)

## Model

In [8]:
!pip install keras_multi_head
#!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-multi-head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14993 sha256=46b421d95386e950c650eb0e66fec82a93fb5970a5db6ebe830cd229526bb330
  Stored in directory: /root/.cache/pip/wheels/86/aa/3c/9d15d24005179dae08ff291ce99c754b296347817d076fd9fb
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=b390405a5fa1716b9095d8f4fe984f46bf8d111e0b1ca8743e1a7197ec28b3f8
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-multi-head keras-self-attention


In [9]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint
from keras_multi_head import MultiHead
#from keras_self_attention import SeqSelfAttention as Attention

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
#current_path ='data/'
#local_path = 'data/'

In [10]:
# set parameters:
# V = 142381
# V = 226928
V = 249274 # 48456 #  33531 6158 48456 249274
embedding_dim = 50
max_len = 1 # 1 #  1945 77 1
filters_num = 64
kernel_size = 3

In [11]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu',
                padding='same')
maxpool1d = MaxPool1D(max_len)
batchnorm1d = BatchNormalization()
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(MultiHead(conv1d))(input_vector)
validation_conv = MultiHead(conv1d)(validation_vector)
convolutional_vector = tf.keras.backend.squeeze(convolutional_vector, axis=4)
validation_conv = tf.keras.backend.squeeze(validation_conv, axis=3)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [12]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 3, 1, 50) (None, 1, 50) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [13]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [14]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [15]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mae', metrics=['mse', 'mae', 'mape'])

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1, 50)    12463700    ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        12463700    ['input_2[0][0]']            

In [17]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 17s 15ms/step - loss: 1.3400 - mse: 4.3216 - mae: 1.3400 - mape: 53.9631 - val_loss: 0.7928 - val_mse: 2.1147 - val_mae: 0.7928 - val_mape: 46.6034
Epoch 2/30
256/256 [==============================] - 3s 14ms/step - loss: 0.7853 - mse: 1.9153 - mae: 0.7853 - mape: 45.1300 - val_loss: 0.8228 - val_mse: 1.9083 - val_mae: 0.8228 - val_mape: 45.0644
Epoch 3/30
256/256 [==============================] - 3s 13ms/step - loss: 0.7018 - mse: 1.5138 - mae: 0.7018 - mape: 38.8020 - val_loss: 0.8493 - val_mse: 1.8585 - val_mae: 0.8493 - val_mape: 44.1024
Epoch 4/30
256/256 [==============================] - 3s 13ms/step - loss: 0.6076 - mse: 1.2430 - mae: 0.6076 - mape: 33.3664 - val_loss: 0.8703 - val_mse: 1.9004 - val_mae: 0.8703 - val_mape: 43.8311
Epoch 5/30
256/256 [==============================] - 3s 13ms/step - loss: 0.5496 - mse: 1.0798 - mae: 0.5496 - mape: 29.7028 - val_loss: 0.8805 - val_mse: 1.9455 - val_mae: 0.8805 - val_mape: 43

In [18]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 10s 4ms/step - loss: 1.0227 - mse: 2.0170 - mae: 1.0227 - mape: 44.8289
[1.0226753950119019, 2.017009973526001, 1.0226753950119019, 44.82888412475586]


In [19]:
# model.save(path+"IMDB_ADLFM_Multi_head_conv1d")